In [ ]:
import pandas as pd
import numpy as np
import os
from os.path import dirname, join
from tkinter import filedialog
from copy import deepcopy
import tifffile
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from shapely.geometry import Point, Polygon

In [ ]:
# parameters
consecutive_window = 3
threshold_alpha = 1
threshold_D = 0.15 # um2/s
threshold_R2 = 0.5